## 패키지 가져오기

In [1]:
from pykrx import stock
import pandas_datareader.data as web
from talib import BBANDS, RSI, MACD

import pandas as pd
import numpy as np

import time
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
from pykrx import stock
from pykrx import bond
import pandas_datareader.data as web
import FinanceDataReader as fdr

from talib import RSI, BBANDS, MACD

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import seaborn as sns

from datetime import datetime
import time
from tqdm import tqdm

import multiprocessing as mp
from multiprocessing import Pool, Manager

import warnings
warnings.filterwarnings('ignore')

In [2]:
num_cores = mp.cpu_count()
print(f"사용가능한 코어 수: {num_cores}")

사용가능한 코어 수: 8


In [3]:
# 2023년 3월 1일 기준 KOSPI와 KOSDAQ의 종목 수를 파악
start_date, end_date = "2015-02-11", "2023-03-01"
kospi_tickers = stock.get_market_ticker_list(end_date, market='KOSPI')
kosdaq_tickers = stock.get_market_ticker_list(end_date, market='KOSDAQ')
all_tickers = kospi_tickers + kosdaq_tickers

print("KOSPI 종목 수: ", len(kospi_tickers))
print("KOSDAQ 종목 수: ", len(kosdaq_tickers))
print("전체 종목 수: ", len(all_tickers))

KOSPI 종목 수:  943
KOSDAQ 종목 수:  1615
전체 종목 수:  2558


## 전략 수립 및 구현

12일을 단기 이동평균, 26일 중기 이동평균으로 사용  
단기 이동평균과 중기 이동평균의 차이를 MACD Signal이라고 하고, MACD Signal의 9일 이동평균선을 사용
만약 MACD가 MACD Signal보다 높으면 매수,낮으면 매도를 실시 함.  
이 때, MACD Signal이 가장 높은 100개(최대)를 매수하고, 가장 낮은 100개(최대)를 매도함  
성과는 연간 -4.7%의 수익률을 기록

In [4]:
rebalance_date = stock.get_market_ohlcv(start_date, end_date, "005930").index
rebalance_date = pd.to_datetime(rebalance_date, format='%Y-%m-%d')
rebalance_date[:10]

DatetimeIndex(['2015-02-11', '2015-02-12', '2015-02-13', '2015-02-16',
               '2015-02-17', '2015-02-23', '2015-02-24', '2015-02-25',
               '2015-02-26', '2015-02-27'],
              dtype='datetime64[ns]', name='날짜', freq=None)

In [5]:
# 쓰이는 모든 티커들을 모음
all_tickers = []
for date in tqdm(rebalance_date):
    kospi_tickers = stock.get_market_ticker_list(date, market='KOSPI')
    kosdaq_tickers = stock.get_market_ticker_list(date, market='KOSDAQ')
    all_tickers += kospi_tickers
    all_tickers += kosdaq_tickers
all_tickers = list(set(all_tickers))
print(f"해당 기간동안의 모든 ticker 개수는 {len(all_tickers)}")

100%|██████████| 1981/1981 [17:56<00:00,  1.84it/s]


해당 기간동안의 모든 ticker 개수는 2863


In [6]:
def return_price_df(tickers, start_date, end_date):
    df = pd.DataFrame()
    for ticker in tqdm(tickers):
        price_df = stock.get_market_ohlcv(start_date, end_date, ticker)['종가'].to_frame(ticker)
        df = pd.concat([df, price_df], axis=1)
    return df

price_df = return_price_df(all_tickers, start_date, end_date)
price_df.to_csv('MACD_price_df.csv')

100%|██████████| 2863/2863 [04:02<00:00, 11.80it/s]


In [7]:
price_df = pd.read_csv('MACD_price_df.csv', index_col=0)
price_df = price_df.fillna(0)
price_df.head()

,900340,192080,000140,036640,095190,002760,000210,069330,036890,067370,...,096640,200670,400760,367480,007630,195990,002960,038950,077360,225440
날짜,,,,,,,,,,,,,,,,,,,,,
2015-02-11,0.0,0.0,12000,3690,4614,696,51777,9980,5440,0.0,...,5333,61089,0.0,0.0,3735.0,0.0,483500,7320,4450,0.0
2015-02-12,0.0,0.0,11800,3685,4641,676,49789,9400,5390,0.0,...,5179,55861,0.0,0.0,3610.0,0.0,469500,7700,4420,0.0
2015-02-13,0.0,0.0,12300,3680,4641,669,50331,9220,5670,0.0,...,5251,59586,0.0,0.0,3520.0,0.0,483000,7670,4340,0.0
2015-02-16,0.0,0.0,12450,3665,4686,680,50693,9000,5950,0.0,...,5240,53784,0.0,0.0,3600.0,0.0,475000,7750,4255,0.0
2015-02-17,0.0,0.0,12000,3605,4695,719,51506,8920,5960,0.0,...,5148,54573,0.0,0.0,3545.0,0.0,475000,7720,4165,0.0


In [8]:
fastperiod = 12
slowperiod = 26
signalperiod = 5

macd_df = price_df.copy()
macd_signal_df = price_df.copy()
macd_hist_df = price_df.copy()

for col in price_df.columns:
    macd_df,macd_signal, macd_hist = MACD(price_df[col], fastperiod=fastperiod, 
                                       slowperiod=slowperiod, signalperiod=signalperiod)
    macd_df[col], macd_signal_df[col], macd_hist_df[col] = macd, macd_signal, macd_hist

NameError: name 'macd' is not defined

In [ ]:
macd_signal_df = macd_signal_df.fillna(0)
def calculate_weights(macd_signal_df, long=100, short=100):
    long_signal = ((macd_signal_df
                  .where(macd_signal_df > 0)
                  .rank(axis=1, ascending=False) <= long)
                  .astype(int))
    short_signal = ((macd_signal_df
                   .where(macd_signal_df < 0)
                   .rank(axis=1, ascending=True) <= short)
                    .astype(int)
                    .mul(-1))
    signal = long_signal.add(short_signal)
    def calculate_weights_from_signal(arr):
        long_signal = (arr == 1).astype(int)
        short_signal = (arr == -1).astype(int)
        long_cnt = np.clip(np.sum((arr == 1)), 0, long)
        short_cnt = np.clip(np.sum((arr == -1)), 0, short)
        if long_cnt > 0:
            long_signal = long_signal / long_cnt
        if short_cnt > 0:
            short_signal = -1 * short_signal / short_cnt
        return long_signal.add(short_signal)
    signal = signal.apply(calculate_weights_from_signal, axis=1)
    return signal
weights = calculate_weights(macd_signal_df)
weights.index = pd.to_datetime(weights.index, format='%Y-%m-%d')

In [ ]:
# 수익률 계산
# NaN 값은 일단 0으로 처리
fwd_return = price_df.pct_change().shift(-1)
fwd_return = fwd_return.fillna(0)
fwd_return.index = pd.to_datetime(fwd_return.index, format='%Y-%m-%d')
fwd_return.tail()

In [ ]:
# 일별 수익률 계산
daily_return = weights.mul(fwd_return).sum(axis=1)
daily_return = daily_return[(daily_return.index >="2015-03-01") & (daily_return.index < "2023-02-28")]
daily_return = daily_return.to_frame("daily_return")
daily_return

In [ ]:
# 누적수익률 계산
cumulative_return = np.exp(np.log(daily_return['daily_return'] + 1).cumsum())-1
cumulative_return = cumulative_return.to_frame("cumulative_return")
cumulative_return

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)
ax.plot(cumulative_return)
ax.set_title("CUMULATIVE RETURN", fontsize=16)
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

## 자세한 결과 분석

In [ ]:
import pyfolio as pf
from pyfolio.plotting import plot_rolling_returns, plot_rolling_sharpe
from pyfolio.timeseries import forecast_cone_bootstrap

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(16, 8))

plot_rolling_returns(daily_return['daily_return'],
                     logy=False,
                     legend_loc='best',
                     volatility_match=False,
                    ax=axes[0])
plot_rolling_sharpe(daily_return['daily_return'], ax=axes[1], rolling_window=63)
axes[0].set_title('Cumulative Returns', fontsize=16)
axes[1].set_title('Rolling Sharpe Ratio (3 Months)', fontsize=16)

In [ ]:
pf.create_returns_tear_sheet(daily_return['daily_return'])

## 전략수정(역으로 해보기)

위의 전략과 반대되는 전략 사용  
MACD Signal이 가장 높은 100개(최대)를 매도하고, 가장 낮은 100개(최대)를 매수함  
성과는 더 안좋음

In [ ]:
price_df = pd.read_csv('MACD_price_df.csv', index_col=0)
price_df = price_df.fillna(0)
price_df.head()

In [ ]:
fastperiod = 12
slowperiod = 26
signalperiod = 5

macd_df = price_df.copy()
macd_signal_df = price_df.copy()
macd_hist_df = price_df.copy()

for col in price_df.columns:
    macd_df,macd_signal, macd_hist = MACD(price_df[col], fastperiod=fastperiod, 
                                       slowperiod=slowperiod, signalperiod=signalperiod)
    macd_df[col], macd_signal_df[col], macd_hist_df[col] = macd, macd_signal, macd_hist

In [ ]:
macd_signal_df = macd_signal_df.fillna(0)
def calculate_weights(macd_signal_df, long=100, short=100):
    long_signal = ((macd_signal_df
                  .where(macd_signal_df < 0)
                  .rank(axis=1, ascending=False) <= long)
                  .astype(int))
    short_signal = ((macd_signal_df
                   .where(macd_signal_df > 0)
                   .rank(axis=1, ascending=True) <= short)
                    .astype(int)
                    .mul(-1))
    signal = long_signal.add(short_signal)
    def calculate_weights_from_signal(arr):
        long_signal = (arr == 1).astype(int)
        short_signal = (arr == -1).astype(int)
        long_cnt = np.clip(np.sum((arr == 1)), 0, long)
        short_cnt = np.clip(np.sum((arr == -1)), 0, short)
        if long_cnt > 0:
            long_signal = long_signal / long_cnt
        if short_cnt > 0:
            short_signal = -1 * short_signal / short_cnt
        return long_signal.add(short_signal)
    signal = signal.apply(calculate_weights_from_signal, axis=1)
    return signal
weights = calculate_weights(macd_signal_df)
weights.index = pd.to_datetime(weights.index, format='%Y-%m-%d')

In [ ]:
# 수익률 계산
# NaN 값은 일단 0으로 처리
fwd_return = price_df.pct_change().shift(-1)
fwd_return = fwd_return.fillna(0)
fwd_return.index = pd.to_datetime(fwd_return.index, format='%Y-%m-%d')
fwd_return.tail()

In [ ]:
# 일별 수익률 계산
daily_return = weights.mul(fwd_return).sum(axis=1)
daily_return = daily_return[(daily_return.index >="2015-03-01") & (daily_return.index < "2023-02-28")]
daily_return = daily_return.to_frame("daily_return")
daily_return

In [ ]:
# 누적수익률 계산
cumulative_return = np.exp(np.log(daily_return['daily_return'] + 1).cumsum())-1
cumulative_return = cumulative_return.to_frame("cumulative_return")
cumulative_return

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)
ax.plot(cumulative_return)
ax.set_title("CUMULATIVE RETURN", fontsize=16)
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))